Với sự phát triển mạnh mẽ và ngày càng tăng của dữ liệu thì dường như Grid Search đã không còn là phương pháp phù hợp cho việc tối ưu hóa model nữa. Thay vào đó là việc áp dụng Random Search hay một kỹ thuật tối ưu khác là **Bayesian** với bayesian sẽ yêu cầu những setup phức tạp hơn so với Random Search hay Grid Search.

Ý tưởng chính của Bayesian Optimization là tối ưu một **proxy function** (surrogate function) - một **function thay thế** hơn là tối ưu objective function (đây cũng là điều mà cả Grid Search và Random Search đều tối ưu). Chúng ta sử dụng Bayesian nếu không có gradients, nếu kiểm tra với objective function thực sự tốn kém (về mặt chi phí) (nếu không tốn kém thì sử dụng Random Search) và search space chứa nhiễu và phức tạp.

Bayesian luôn cân bằng giữa khám phá - *exploration* và khai thác - *exploitation*. Ban đầu, Bayesian sẽ cố gắng khám phá dữ liệu một cách ngẫu nhiên, do đó sẽ sử dụng proxy function. Dựa trên proxy function, Bayesian sẽ khởi tạo **approximate knowledge** để lấy mẫu các sample data và minimize cost function. Sau đó sử dụng **priors param** để đưa ra quyết định hiệu quả cho việc sampling data. Nhờ đó quá trình minimization sẽ được diễn ra nhanh hơn bằng cách giới hạn số lượng đánh giá cần phải thực hiện.

Bayesian Optimization sử dụng **acquisition function** để mô tả cho chúng ta biết làm thế nào để thực hiện trên một observation data. Để có thể quản lý **trade-off giữa exploration và exploitation**, Bayesian xác định acquisition function cung cấp một phép đo thực nghiệm trên các điểm dữ liệu.

Thông thường Bayesian optimization được thể hiện bởi Gaussian process - và quá trình này sẽ thực hiện tốt khi search space không quá phức tạp, smooth và có thể nắm bắt hay đoán trước được. Để có thể tìm kiếm trên search space phức tạp hơn chúng ta có thể sử dụng **tree algorithms** (random forest) hoặc một cách tiếp cận khác là  Tree Parzen
Estimators hay Tree-structured Parzen Estimators (TPEs)

Thay vì xây dựng một model để đánh giá score giữa tập hợp các parameters thì TPE ước lượng các parameter theo phân phối đa biến xác định giá trị các parameter tốt nhất dựa trên **successive approximations** có được qua những lần thực nghiệm. Bằng cách này TPE lấy tập hợp các parameter tốt nhất bằng cách lấy mẫu chúng từ một phân phối xác suất.

In [1]:
!gdown --id 1-YYnr636nn1vAOU1nwgbBajY7wF6s75j
!gdown --id 1qMMdfMkvw-tbmlX6b-M9IKkdCXWr5Zy9

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-YYnr636nn1vAOU1nwgbBajY7wF6s75j
To: /content/train.csv
100% 93.7M/93.7M [00:01<00:00, 52.5MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qMMdfMkvw-tbmlX6b-M9IKkdCXWr5Zy9
To: /content/test.csv
100% 58.9M/58.9M [00:00<00:00, 92.1MB/s]


In [2]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


In [3]:
!pip install ipython-autotime
%load_ext autotime

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
time: 438 µs (started: 2024-09-25 17:45:58 +00:00)


In [4]:
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from functools import partial

import warnings
warnings.filterwarnings("ignore")

import lightgbm as lgb
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer

time: 5.87 s (started: 2024-09-25 17:45:58 +00:00)


In [5]:
X = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")

X.head(2)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233


time: 3.98 s (started: 2024-09-25 17:46:22 +00:00)


In [6]:
X_test.head(2)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,B,B,B,C,B,B,A,E,E,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,5,A,B,A,C,B,C,A,E,C,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940


time: 19.8 ms (started: 2024-09-25 17:46:26 +00:00)


In [7]:
y = X.target
X = X.set_index('id').drop('target', axis='columns')
X_test = X_test.set_index('id')

time: 118 ms (started: 2024-09-25 17:46:26 +00:00)


In [8]:
# processs data
categoricals = [item for item in X.columns if 'cat' in item]
cat_values = np.unique(X[categoricals].values)
cat_dict = dict(zip(cat_values, range(len(cat_values))))

X[categoricals] = X[categoricals].replace(cat_dict).astype('category')
X_test[categoricals] = X_test[categoricals].replace(cat_dict).astype('category')

time: 6.59 s (started: 2024-09-25 17:46:38 +00:00)


Sklearn callback

In [13]:
def report_perf(optimizer, X, y, title="model", callbacks=None):
    start = time()

    if callbacks is not None:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)

    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_

    print((title + " use %.2f seconds,  hyperparameters checked: %d, best CV score: %.3f "+ u"\u00B1"+" %.3f") % (time() - start,
                                   len(optimizer.cv_results_['params']),
                                   best_score,
                                   best_score_std))
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

time: 910 µs (started: 2024-09-25 18:10:51 +00:00)


In [14]:
scoring = make_scorer(partial(mean_squared_error, squared=False), greater_is_better=False)
kf = KFold(n_splits=5, shuffle=True, random_state=0)
reg = lgb.LGBMRegressor(boosting_type='gbdt',
                        metric='rmse',
                        objective='regression',
                        n_jobs=1,
                        verbose=-1,
                        random_state=0)

time: 1.79 ms (started: 2024-09-25 18:10:52 +00:00)


search space thường lấy theo chuẩn phân phối hoặc log

In [15]:
search_spaces = {
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'n_estimators': Integer(30, 5000),                   # number of tree
    'num_leaves': Integer(2, 512),                       # number of node (leaf)
    'max_depth': Integer(-1, 256),                       # max_depth of tree
    'min_child_samples': Integer(1, 256),                # min data in each leaf
    'max_bin': Integer(100, 1000),                       # max number of bin store eigenvalues
    'subsample': Real(0.01, 1.0, 'uniform'),             # sample data ratio of training
    'subsample_freq': Integer(0, 10),                    # <=0 not active
    'colsample_bytree': Real(0.01, 1.0, 'uniform'),      # sample data ratio of tree
    'min_child_weight': Real(0.01, 10.0, 'uniform'),     # min weight of node
    'reg_lambda': Real(1e-9, 100.0, 'log-uniform'),      # L2
    'reg_alpha': Real(1e-9, 100.0, 'log-uniform'),       # L1
   }

time: 13.1 ms (started: 2024-09-25 18:10:54 +00:00)


In [16]:
opt = BayesSearchCV(estimator=reg,
                    search_spaces=search_spaces,
                    scoring=scoring,
                    cv=kf,
                    n_iter=60,
                    n_points=3,                                       # number of hyperparameter sets to evaluate simultaneously
                    n_jobs=-1,
                    iid=False,                                        # if False => optimize base on cv
                    return_train_score=False,
                    refit=False,
                    optimizer_kwargs={'base_estimator': 'GP'},        # GP - Gaussian Process: optimize parameter
                    random_state=0)

time: 3.88 ms (started: 2024-09-25 18:10:55 +00:00)


In [ ]:
overdone_control = DeltaYStopper(delta=0.0001)               # if optimized gain => stop
time_limit_control = DeadlineStopper(total_time=60 * 60 * 1) # limit 1h => stop

best_params = report_perf(opt, X, y,'LGBMR_regression',
                          callbacks=[overdone_control, time_limit_control])

LGBMR_regression use 3225.32 seconds,  hyperparameters checked: 3, best CV score: -0.726 ± 0.002
Best parameters:
OrderedDict([('colsample_bytree', 0.5356545233026259),
             ('learning_rate', 0.22498782522548938),
             ('max_bin', 660),
             ('max_depth', 165),
             ('min_child_samples', 220),
             ('min_child_weight', 9.805757064152607),
             ('n_estimators', 667),
             ('num_leaves', 3),
             ('reg_alpha', 0.0027950270441846913),
             ('reg_lambda', 4.6259107779014013e-07),
             ('subsample', 0.07025834820938834),
             ('subsample_freq', 5)])

time: 53min 45s (started: 2024-09-25 18:10:56 +00:00)


In [ ]:
# train with best params
reg = lgb.LGBMRegressor(boosting_type='gbdt',
                        metric='rmse',
                        objective='regression',
                        n_jobs=1,
                        verbose=-1,
                        random_state=0,
                         **best_params)

time: 5.71 ms (started: 2024-09-25 19:04:42 +00:00)


In [ ]:
reg.fit(X, y)


LGBMRegressor(colsample_bytree=0.5356545233026259,
              learning_rate=0.22498782522548938, max_bin=660, max_depth=165,
              metric='rmse', min_child_samples=220,
              min_child_weight=9.805757064152607, n_estimators=667, n_jobs=1,
              num_leaves=3, objective='regression', random_state=0,
              reg_alpha=0.0027950270441846913,
              reg_lambda=4.6259107779014013e-07, subsample=0.07025834820938834,
              subsample_freq=5, verbose=-1)

time: 9.74 s (started: 2023-05-31 16:25:34 +00:00)
